In [1]:
import findspark

In [2]:
# spark가 설치된 위치를 찾아준다
findspark.init('/opt/spark')

In [3]:
import pyspark
from pyspark.sql.streaming import DataStreamReader

In [4]:
# mysql connector 패키지와 함께 스파크를 실행시킨다
spark = pyspark.sql.SparkSession.builder.appName('myFirstApp') \
.master('spark://master:7077') \
.config('spark.jars.packages','org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2') \
.getOrCreate()

:: loading settings :: url = jar:file:/opt/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1c0cec11-3419-4ae9-a41c-c7346d1c21b1;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 600ms :: artifacts dl 13ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from central in [default]


In [5]:
# 실시간으로 kafka에 저장된 데이터를 읽어올 수 있는 readStream 생성
df = spark.readStream.format("kafka").option("kafka.bootstrap.servers",'172.17.0.6:9092,172.17.0.7:9092,172.17.0.8:9092').option("subscribe","hdtest").load()

In [6]:
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [7]:
# 기본적으로 kafka는 value가 binary type으로 들어오므로 string으로 타입변환시켜준다
df = df.selectExpr("CAST(value AS STRING)")

In [8]:
df.printSchema()

root
 |-- value: string (nullable = true)



In [9]:
# readStream이 실행됨과 동시에 writeStream이 실행되어 5초마다 한번씩 hdfs에 데이터를 저장함
df.writeStream.trigger(processingTime='5 seconds').outputMode("append").format("json") \
.option("path", "/streaming/out").option("checkpointLocation", "/streaming/checkpointLocation").start().awaitTermination()

2021-08-20 15:31:00,145 WARN streaming.ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 20141 milliseconds


KeyboardInterrupt: 